## Serializing and Deserializing Strings

In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions

### Topics

In [ ]:
topics = ["string_topic", "avro_topic"]
brokers = "kafka1:9092,kafka2:9093,"

### Preparing the field

In [ ]:
a = AdminClient(conf = {'bootstrap.servers': brokers})

In [ ]:
new_topics = [NewTopic(topic, num_partitions=2, 
        replication_factor=1) for topic in topics]

In [ ]:
fs = a.create_topics(new_topics)
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(topic))
    except Exception as e:
        print("Failed to create topic {}: {}".format(topic, e))

### Producer

In [ ]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
import json
from uuid import uuid4

In [ ]:
producer_conf = {
        'bootstrap.servers': brokers,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')}

In [ ]:
producer = SerializingProducer(producer_conf)

In [ ]:
with open('./data.json') as f:
    data = json.load(f)
data

In [ ]:
for m in data:
    print(m)
    producer.produce(topic=topics[0],key=str(uuid4()),value=str(m))
    producer.poll(0)

In [ ]:
producer.flush()

### Consumer

In [ ]:
from confluent_kafka import DeserializingConsumer
from confluent_kafka.serialization import StringDeserializer

In [ ]:
string_deserializer = StringDeserializer('utf_8')

In [ ]:
consumer_conf = {
    'bootstrap.servers': brokers,
    'key.deserializer': string_deserializer,
    'value.deserializer': string_deserializer,
    'group.id': str(uuid4()),
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'}

In [ ]:
consumer = DeserializingConsumer(consumer_conf)
consumer.subscribe([topics[0]])
topics[0]

In [ ]:
try:
    while True:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            print("{} [{}] at offset  {} with key  {}:  {}".format(msg.topic(), msg.partition(), msg.offset(), str(msg.key()), str(msg.value())))
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
finally:
    # Close down consumer to commit final offsets.
    consumer.close()

## Avro

In [ ]:
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
import random, time
from uuid import uuid4

### Schema Registry

In [ ]:
schema_registry_conf = {'url': "http://schema-registry:8081"}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

schema_registry_client.get_subjects()

In [ ]:
value_schema = open("observation.avsc").read()
value_schema

In [ ]:
avro_serializer = AvroSerializer(value_schema, schema_registry_client)

producer_conf = {'bootstrap.servers': brokers,
                 'key.serializer': StringSerializer('utf_8'),
                 'value.serializer': avro_serializer,
                 'partitioner': 'murmur2_random',
}

producer = SerializingProducer(producer_conf)

In [ ]:
rooms = ["room1", "room2", "room3", "room4"]

In [ ]:
for i in range(1,10):
    value = {"id": i, 
               "value": random.uniform(0,50), 
               "measurement":"temperature", 
               "timestamp": round(time.time()*1000)}
    key = rooms[ random.randint(0,3)]
    print(key + " " + str(value))
    producer.poll(0)
    producer.produce(topic=topics[1], value=value, key=key)
    producer.flush(1)


In [ ]:
schema_registry_client.get_subjects()

In [ ]:
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry.avro import AvroDeserializer

In [ ]:
avro_deserializer = AvroDeserializer(value_schema,schema_registry_client)

string_deserializer = StringDeserializer('utf_8')

In [ ]:
consumer_conf = {'bootstrap.servers': brokers,
                     'key.deserializer': string_deserializer,
                     'value.deserializer': avro_deserializer,
                     'group.id': str(uuid4()),
                     'auto.offset.reset': "earliest"}

consumer = DeserializingConsumer(consumer_conf)
consumer.subscribe([topics[1]])

In [ ]:
while True:
    try:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        m = msg.value()
        if m is not None:
            print(m)
    except KeyboardInterrupt:
        break
        consumer.close()

#### What if we do not know the schema?

We can't see this in python because of a bug of the Avro library. However, we can observe it from [Java](link to the other repo class)